**Section 1: Load Training dataset**

In [ ]:
# Install libraries
!pip install tweet-preprocessor
!pip install transformers==4.28.0
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load general libraries
import numpy as np
import pandas as pd
import torch

# Load preprocessing libraries
import preprocessor as p
import unicodedata as ud
from sklearn.model_selection import train_test_split

# Load embeddings libraries
from transformers import BertTokenizerFast

# Load classifier libraries
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Retrieve 2015 dataset from Github (!!!!!should probably change link at some point)
url = "https://raw.githubusercontent.com/dimosbele/sentiment_analysis_greek/master/modeling/data/df_final.pkl"
dataset = pd.read_pickle(url)

X = np.array(dataset["Text"])
y = np.array(dataset["Sentiment"])

print(X[0])
print(y[0])

Επιχείρηση Βολευτική έδρα!  Βασίλης Οικονόμου:  Από το ΠΑΣΟΚ διαγράφηκε απο το ΓΑΠ επειδη ΔΕΝ ψήφισε το Μνημόνιο... http://t.co/gPgr1Caqxv
-1


**Section 2: Preprocess tweets**

In [ ]:
# Preprocess tweets
def preprocess_tweets(X):
    X = clean_text(X)
    return X

def clean_text(X):
    X = [tweet.lower() for tweet in X]

    # Remove URLS, mentions
    p.set_options(p.OPT.URL, p.OPT.MENTION)
    X = [p.clean(tweet) for tweet in X]

    # Remove Greek accents
    d = {ord('\N{COMBINING ACUTE ACCENT}'):None}
    X = [ud.normalize('NFD',word).translate(d) for word in X]

    return X

for sample in X[:5]:
    print(sample)
print()

X = preprocess_tweets(X)

for sample in X[:5]:
    print(sample)

Επιχείρηση Βολευτική έδρα!  Βασίλης Οικονόμου:  Από το ΠΑΣΟΚ διαγράφηκε απο το ΓΑΠ επειδη ΔΕΝ ψήφισε το Μνημόνιο... http://t.co/gPgr1Caqxv
Κ ά β ο ς: Ψηφοδέλτιο ΣΥΡΙΖΑ Κορινθίας - Πού πας,ΣΥΡΙΖΑ, με δράμι μυαλό... http://t.co/8eIiKYS7CP
@Dora_Bakoyannis Αυτο-καρφώνεσαι όταν αναρωτιέσαι ποιον ΠτΔ θα προτείνει ο ΣΥΡΙΖΑ  αποδεχόμενη την νίκη του αντιπάλου.
Κόμμα «σκαντζόχοιρος»: «Σφαγή» πασοκογενών από τις λίστες του ΣΥΡΙΖΑ http://t.co/rC7EY67qFY
Και γαμώ και την Νέα Δημοκρατία γιατί ειμαι πολιτικο ακκάουντ στην τελική όχι κλαψομούνικο!!!!

επιχειρηση βολευτικη εδρα! βασιλης οικονομου: απο το πασοκ διαγραφηκε απο το γαπ επειδη δεν ψηφισε το μνημονιο...
κ α β ο ς: ψηφοδελτιο συριζα κορινθιας - που πας,συριζα, με δραμι μυαλο...
αυτο-καρφωνεσαι οταν αναρωτιεσαι ποιον πτδ θα προτεινει ο συριζα αποδεχομενη την νικη του αντιπαλου.
κομμα «σκαντζοχοιρος»: «σφαγη» πασοκογενων απο τις λιστες του συριζα
και γαμω και την νεα δημοκρατια γιατι ειμαι πολιτικο ακκαουντ στην τελικη οχι κλαψομουνικο!!!!


In [ ]:
for label in y[:5]:
    print(label)
print()

# Change labels from [-1, 0, 1] to [0, 1, 2]
for i in range(len(y)):
    y[i] += 1

for label in y[:5]:
    print(label)

# Split dataset to train-val-test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

X_train, X_val , y_train, y_val = train_test_split(X_train, y_train, test_size = 0.125, random_state=42, stratify=y_train)

print("Training samples:", len(y_train))
print("Validation samples:", len(y_val))
print("Test samples:", len(y_test))

-1
-1
1
0
-1

0
0
2
1
0
Training samples: 1148
Validation samples: 164
Test samples: 328


In [ ]:
# Load active training samples
X2 = pd.read_csv("drive/MyDrive/Active_Learning/Active_Learning.csv")
X2 = X2.to_numpy().ravel()

y2 = pd.read_csv("drive/MyDrive/Active_Learning/Active_Learning_ground_truth.csv")
y2 = y2.to_numpy().ravel()

for i in range(5):
    print(X2[i])
    print(y2[i])
    print()

# Split dataset to train-val-test (70-10-20)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 0.2, random_state=42, stratify=y2)

X_train2, X_val2 , y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size = 0.125, random_state=42, stratify=y_train2)

#Stack
X_train = np.append(X_train, X_train2)
X_val = X_val2
X_test_old = X_test
X_test = X_test2

y_train = np.append(y_train, y_train2)
y_val = y_val2
y_test_old = y_test
y_test = y_test2

print("Training samples:", len(y_train))
print("Validation samples:", len(y_val))
print("Old test samples:", len(y_test_old))
print("Old test samples:", len(X_test_old))
print("Test samples:", len(X_test))
print("Test samples:", len(y_test))

ως ελευθερη επαγγελματιας, εχω βιωσει απο πρωτο χερι τη λεηλασια που εχουν υποστει η μεσαια ταξη και τα χαμηλα εισοδηματα! γνωριζω καλα τον ανισο αγωνα που δινουν καθημερινα οι πολιτες αυτης της χωρας! στις #εκλογες_2023 δωστε μου φωνη για να γινω η δικη σας! #δικαιοσυνηπαντου
1

σημερα στις 18:00, ο λογος δινεται στους υποψηφιους της #συμμαχιας_ανατροπης.. και ακομη ποια γνωστα γνωστα προσωπα και κινηματα στηριζουν τη συμμαχια. ζωντανα και αποκλειστικα στο #εκλογες_2023 #εκλογες_21__μαιου #υποκλοπες
1

διαχειριση ηττας loading… #εκλογες_2023
1

👷‍♀️ εργατικα ατυχηματα 🔸 72% αυξηση, την περιοδο μητσοτακη στον αριθμο των εργατικων ατυχηματων (θανατηφορων η με σωματικη βλαβη) 📈+16% η αυξηση στα θανατηφορα εργατικα δυστυχηματα πηγη: ελληνικη αστυνομια (στατιστικη επετηριδα) 🗳️ #εκλογες2023 #εκλογες_2023…
0

δειτε το live εδω #εθνικο_κομμα_ελληνες #εκλογες_2023 #εκλογες_21ης__μαιου #δικαιοσυνη
1

Training samples: 1213
Validation samples: 10
Old test samples: 328
Old test samples: 328
Test

**Section 3: Create word embeddings**

In [ ]:
# BERT embeddings
tokenizer = BertTokenizerFast.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
bert_train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
bert_val_encodings = tokenizer(list(X_val), truncation=True, padding=True)
bert_test_old_encodings = tokenizer(list(X_test_old), truncation=True, padding=True)
bert_test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


**Section 4: Model Training**

In [ ]:
# Greek BERT
import evaluate
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    return results

# Create BERT datasets
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def train_BERT():
    bert_train_dataset = TwitterDataset(bert_train_encodings, y_train)
    bert_val_dataset = TwitterDataset(bert_val_encodings, y_val)
    bert_test_old_dataset = TwitterDataset(bert_test_old_encodings, y_test_old)
    bert_test_dataset = TwitterDataset(bert_test_encodings, y_test)

    model = BertForSequenceClassification.from_pretrained("drive/MyDrive/BERT", num_labels=3)

    # Set training arguments
    training_args = TrainingArguments(
        evaluation_strategy='epoch',     # evaluation frequency
        save_strategy='epoch',           # model checkpoint frequency
        logging_strategy='epoch',        # logging frequency
        log_level='warning',             # logging level
        output_dir='./results',          # output directory
        num_train_epochs=6,              # total number of training epochs
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=64,   # batch size for evaluation, change according to GPU memory capabilities
        warmup_steps=200,                # number of warmup steps for learning rate scheduler
        weight_decay=0.001,              # strength of weight decay
        logging_dir='./logs',             # directory for storing logs
        load_best_model_at_end = True,
        metric_for_best_model = "f1",
        seed=42
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=bert_train_dataset,
        eval_dataset=bert_val_dataset,
        compute_metrics=compute_metrics
    )

    metrics=trainer.evaluate(eval_dataset=bert_test_old_dataset)
    print(metrics)

    metrics=trainer.evaluate(eval_dataset=bert_test_dataset)
    print(metrics)

    trainer.train()

    metrics=trainer.evaluate(eval_dataset=bert_test_old_dataset)
    print(metrics)

    metrics=trainer.evaluate(eval_dataset=bert_test_dataset)
    print(metrics)

    trainer.save_model("drive/MyDrive/BERT_Active")

In [ ]:
train_BERT()

{'eval_loss': 0.8366826772689819, 'eval_accuracy': 0.8780487804878049, 'eval_recall': 0.7745865587614356, 'eval_precision': 0.7649431609827649, 'eval_f1': 0.7687175372661628, 'eval_runtime': 1.8205, 'eval_samples_per_second': 180.169, 'eval_steps_per_second': 3.296}
{'eval_loss': 1.5197800397872925, 'eval_accuracy': 0.47368421052631576, 'eval_recall': 0.5833333333333334, 'eval_precision': 0.5222222222222223, 'eval_f1': 0.48034188034188025, 'eval_runtime': 0.1027, 'eval_samples_per_second': 185.032, 'eval_steps_per_second': 9.739}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.108600,1.729968,0.700000,0.611111,0.875000,0.645299
2,0.118700,1.893996,0.500000,0.433333,0.357143,0.388889
3,0.086000,1.088035,0.700000,0.655556,0.711111,0.666667
4,0.038100,1.792075,0.600000,0.544444,0.683333,0.544444
5,0.021400,1.841515,0.700000,0.700000,0.666667,0.674603
6,0.012000,1.746059,0.700000,0.700000,0.666667,0.674603


{'eval_loss': 0.9866450428962708, 'eval_accuracy': 0.875, 'eval_recall': 0.7502345765892565, 'eval_precision': 0.7739867468022807, 'eval_f1': 0.7605667527955612, 'eval_runtime': 1.0522, 'eval_samples_per_second': 311.733, 'eval_steps_per_second': 5.702, 'epoch': 6.0}
{'eval_loss': 1.4354480504989624, 'eval_accuracy': 0.7368421052631579, 'eval_recall': 0.6166666666666667, 'eval_precision': 0.7008547008547009, 'eval_f1': 0.6350931677018633, 'eval_runtime': 0.1089, 'eval_samples_per_second': 174.482, 'eval_steps_per_second': 9.183, 'epoch': 6.0}
